In [ ]:
# | default_exp tools.mcp_tools

In [27]:
# | export
import subprocess
import json
from typing import Dict, Any, List
from pathlib import Path
from agentic.tools.base import BaseTool, ToolMetadata, ToolCategory, create_success_response, create_error_response, extract_validation_error, create_success_response, create_error_response
from agentic.configs.loader import get_mcp_config
from pprint import pprint
from jsonschema import validate, ValidationError  

import logging

# Optional: better logs
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)


"""{
    "type": "object",
    "additionalProperties": False,
    "required": ["tool_name", "arguments"],
    "properties": {
        "tool_name": {
            "type": "string",
            "enum": [],  # Will be filled dynamically: ["filesystem.read_file", "thinking.sequentialthinking"]
            "description": "Name of the tool to call. Use the exact name from the list."
        },
        "arguments": {
            "type": "object",
            "description": "Parameters for the selected tool. Must match the schema below.",
            "anyOf": []  # Will contain one object per tool with: properties, required, etc.
        }
    },
    "$comment": "Dynamically populated by MCPTool.get_parameters_schema()"
}"""

In [28]:
# | export
# --------------------------------------------------------------------------- #
#  MCPTool – now with LOCAL VALIDATION
# --------------------------------------------------------------------------- #
class MCPTool(BaseTool):
    """Generic MCP tool that can work with any MCP server"""

    def __init__(self, server_name: str, server_config: Dict[str, Any]):
        metadata = ToolMetadata(
            name=f"mcp_{server_name}",
            description=f"MCP tool for {server_name} server",
            category=ToolCategory.MCP
        )
        super().__init__(metadata)
        self.server_name = server_name
        self.server_config = server_config
        self.mcp_cmd = server_config["command"] + server_config.get("args", [])
        self._tools_cache = None
        self._schemas_cache = {}

    # --------------------------------------------------------------------- #
    #  1. List tools from MCP server (unchanged)
    # --------------------------------------------------------------------- #

    def _get_mcp_tools(self) -> Dict[str, Any]:
        if self._tools_cache is not None:
            return self._tools_cache
    
        request = {"jsonrpc": "2.0", "id": 1, "method": "tools/list"}
        log.info(f"Fetching tools from MCP server: {' '.join(self.mcp_cmd)}")
        log.debug(f"Request: {request}")
    
        try:
            result = subprocess.run(
                self.mcp_cmd,
                input=json.dumps(request) + "\n",
                capture_output=True,
                text=True,
                timeout=15  # Increased slightly
            )
    
            # CASE 1: Process failed to run
            if result.returncode != 0:
                error_msg = (
                    f"MCP server failed to start or crashed (exit code {result.returncode})\n"
                    f"Command: {' '.join(self.mcp_cmd)}\n"
                    f"STDOUT: {result.stdout.strip()}\n"
                    f"STDERR: {result.stderr.strip()}"
                )
                log.error(error_msg)
                print("\nMCP SERVER ERROR:")
                print(error_msg)
                return {}
    
            # CASE 2: Empty or invalid JSON output
            stdout = result.stdout.strip()
            if not stdout:
                error_msg = (
                    "MCP server returned empty response\n"
                    f"Command: {' '.join(self.mcp_cmd)}\n"
                    f"STDERR: {result.stderr.strip()}"
                )
                log.error(error_msg)
                print("\nMCP EMPTY RESPONSE:")
                print(error_msg)
                return {}
    
            try:
                response = json.loads(stdout)
            except json.JSONDecodeError as e:
                error_msg = (
                    f"Invalid JSON from MCP server:\n"
                    f"Raw output: {stdout}\n"
                    f"Parse error: {e}\n"
                    f"STDERR: {result.stderr.strip()}"
                )
                log.error(error_msg)
                print("\nMCP JSON PARSE ERROR:")
                print(error_msg)
                return {}
    
            # CASE 3: JSON-RPC error
            if "error" in response:
                err = response["error"]
                error_msg = (
                    f"MCP server returned JSON-RPC error:\n"
                    f"Code: {err.get('code')}, Message: {err.get('message')}\n"
                    f"Full response: {response}\n"
                    f"STDERR: {result.stderr.strip()}"
                )
                log.error(error_msg)
                print("\nMCP JSON-RPC ERROR:")
                print(error_msg)
                return {}
    
            # CASE 4: Missing expected fields
            if "result" not in response or "tools" not in response["result"]:
                error_msg = (
                    f"Unexpected response format from MCP server:\n"
                    f"Response: {response}\n"
                    f"Expected: {{'result': {{'tools': [...]}}}}"
                )
                log.error(error_msg)
                print("\nMCP INVALID FORMAT:")
                print(error_msg)
                return {}
    
            # SUCCESS
            tools = response["result"]["tools"]
            self._tools_cache = {tool["name"]: tool for tool in tools}
            log.info(f"Loaded {len(tools)} tools: {', '.join(self._tools_cache.keys())}")
            return self._tools_cache
    
        except subprocess.TimeoutExpired:
            error_msg = (
                f"MCP server timed out after 15s\n"
                f"Command: {' '.join(self.mcp_cmd)}"
            )
            log.error(error_msg)
            print("\nMCP TIMEOUT:")
            print(error_msg)
            return {}
    
        except Exception as e:
            error_msg = f"Unexpected error while fetching tools: {type(e).__name__}: {e}"
            log.error(error_msg)
            print("\nMCP UNEXPECTED ERROR:")
            print(error_msg)
            return {}

    # --------------------------------------------------------------------- #
    #  2. Build compact schema (nested + flat) – **cleaned for LLMs**
    # --------------------------------------------------------------------- #
    def get_parameters_schema(self) -> Dict[str, Any]:
        if self.server_name in self._schemas_cache:
            return self._schemas_cache[self.server_name]

        tools = self._get_mcp_tools()
        # pprint(tools)   

        # ---------- Fallback ----------
        if not tools:
            fallback = {
                "type": "object",
                "properties": {
                    "tool_name": {"type": "string", "description": "Name of the MCP tool"},
                    "arguments": {"type": "object", "description": "Arguments for the tool"}
                },
                "required": ["tool_name", "arguments"],
                "additionalProperties": False
            }
            self._schemas_cache[self.server_name] = fallback
            return fallback

        # ---------- Walk nested tools ----------
        leaf_tools: Dict[str, Dict] = {}   # full_name -> inputSchema

        def _walk(current_path: str, node: Dict) -> None:
            if "inputSchema" in node:
                schema = node["inputSchema"].copy()
                # Strip meta keys that validators/LLMs ignore
                for meta in ("$schema", "$id", "$comment"):
                    schema.pop(meta, None)
                schema.setdefault("additionalProperties", False)
                schema.setdefault("type", "object")
                leaf_tools[current_path] = schema
                return
            for sub_name, sub_node in node.get("properties", {}).items():
                new_path = f"{current_path}.{sub_name}" if current_path else sub_name
                _walk(new_path, sub_node)

        for name, info in tools.items():
            _walk(name, info)

        # ---------- Build final schema ----------
        tool_names = []
        argument_schemas = []

        for full_name, schema in leaf_tools.items():
            tool_names.append(full_name)
            argument_schemas.append(schema)

        schema = {
            "type": "object",
            "properties": {
                "tool_name": {
                    "type": "string",
                    "enum": tool_names,
                    "description": f"Available tools: {', '.join(tool_names)}"
                },
                "arguments": {
                    "type": "object",
                    "description": "Exact arguments required by the selected tool",
                    "anyOf": argument_schemas
                }
            },
            "required": ["tool_name", "arguments"],
            "additionalProperties": False
        }

        self._schemas_cache[self.server_name] = schema
        return schema

    # --------------------------------------------------------------------- #
    #  3. Execute with **LOCAL VALIDATION**
    # --------------------------------------------------------------------- #
    def execute(self, tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
        """Execute any MCP tool – validates input **before** calling the server."""
        # ---- 1. Build the instance to validate ----
        instance = {
            "tool_name": tool_name,
            "arguments": arguments
        }

        # ---- 2. Get the schema (cached) ----
        schema = self.get_parameters_schema()

        # ---- 3. VALIDATE locally ----
        try:
            validate(instance=instance, schema=schema)
        except ValidationError as e:
            # Human-readable error (shows exact field + why)
            path = " -> ".join(str(p) for p in e.absolute_path) or "root"
            return create_error_response(
                f"Validation failed for '{tool_name}': {e.message} (at {path})"
            )

        # ---- 4. If valid → call MCP server ----
        try:
            request = {
                "jsonrpc": "2.0",
                "id": 1,
                "method": "tools/call",
                "params": {
                    "name": tool_name,
                    "arguments": arguments
                }
            }

            result = subprocess.run(
                self.mcp_cmd,
                input=json.dumps(request) + "\n",
                capture_output=True,
                text=True,
                timeout=30
            )

            if result.returncode == 0:
                response = json.loads(result.stdout.strip())
                if "result" in response:
                    return create_success_response(
                        f"MCP {self.server_name} tool '{tool_name}' executed successfully",
                        data={
                            "server": self.server_name,
                            "tool": tool_name,
                            "result": response["result"]
                        }
                    )

            return create_error_response(f"MCP server error: {result.stderr}")

        except Exception as e:
            return create_error_response(f"Error calling MCP {self.server_name}: {str(e)}")

In [29]:
# | export
class MCPToolFactory:
    """Factory to create MCP tools from config"""
    
    @staticmethod
    def load_mcp_tools_from_config() -> List[BaseTool]:
        """Load all MCP tools from config.toml"""
        try:
            mcp_config = get_mcp_config()
            
            
            tools = []
            for server_config in mcp_config.servers:
                server_name = server_config["name"]
                tool = MCPTool(server_name, server_config)
                tools.append(tool)
            
            return tools
            
        except Exception as e:
            print(f"Error loading MCP tools from config: {e}")
            return []

In [30]:
# Example usage
mcp_tools = MCPToolFactory.load_mcp_tools_from_config()
from pprint import pprint
schema = mcp_tools[0].get_parameters_schema()
# pprint(schema)

INFO:__main__:Fetching tools from MCP server: docker run -i --rm --volume=/home/pranav-pc/projects/applied-GenAI-lab:/project ghcr.io/mark3labs/mcp-filesystem-server:latest /project
INFO:__main__:Loaded 14 tools: copy_file, create_directory, delete_file, get_file_info, list_allowed_directories, list_directory, modify_file, move_file, read_file, read_multiple_files, search_files, search_within_files, tree, write_file
